In [1]:
import openpyxl
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import basedosdados as bd
import numpy as np


In [2]:
os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS E CONHECIMENTO\\415 - Repositório de Dados\\Repositório Local\\República")

In [3]:
df = pd.read_excel("assedio_estados.xlsx")

In [4]:
df

,UF,Sigla_uf,proibicao_assedio_moral_sexual_rju_lei,conceito_moral_sexual_rju_lei,Regime Jurídico Único,Lei esparsa encontrada,Alterações
0,Acre,AC,Apenas assédio Moral no RJU,Apenas conceito de assédio moral no RJU,"Lei Complementar n.º 39, de 29 de dezembro de ...",-,NaN
1,Alagoas,AL,Sem previsão legislativa,Sem conceito para assédio no RJU,Lei n.º 5.247 de 26 de julho de 1991,-,NaN
2,Amapá,AP,Sem previsão legislativa,Sem conceito para assédio no RJU,"Lei n.º 0066, de 03 de maio de 1993",-,NaN
3,Amazonas,AM,Apenas assédio sexual em lei esparsa,Apenas conceito de assédio sexual em lei esparsa,Lei n.º 1.762 de 17 de novembro de 1986,"Lei n.º 5.378, de 6 de janeiro de 2021.",NaN
4,Bahia,BA,Sem previsão legislativa,Sem conceito para assédio no RJU,Lei n.º 6.677 de 26 de setembro de 1994,-,NaN
5,Ceará,CE,Apenas assédio Moral no RJU,Apenas conceito de assédio moral em lei esparsa,"Lei n.º 9.826, de 14 de maio de 1974",Lei n.º 15.036 de 18 de novembro de 2011,NaN
6,Distrito Federal,DF,Ambos no RJU,Sem conceito para assédio no RJU,"Lei complementar n.º 840, de 23 de dezembro de...",-,NaN
7,Espírito Santo,ES,Sem previsão legislativa,Sem conceito para assédio no RJU,"Lei Complementar n.º 46, de 31 de janeiro de 1994",-,NaN
8,Goiás,GO,Ambos no RJU,Apenas conceito de assédio moral em lei esparsa,"Lei n.º 20.756, de 28 de janeiro de 2020","Lei nº 18.456, de 30 de abril de 2014.",NaN
9,Maranhão,MA,Sem previsão legislativa,Sem conceito para assédio no RJU,"Lei n.º 6.107, de 27 de julho de 1994",-,NaN


In [ ]:
## Adicionando atualização de Pernambuco: 

In [7]:
df['proibicao_assedio_moral_sexual_rju_lei'][16]="Ambos em lei esparsa"

In [9]:
client = bigquery.Client()
dataset_ref = client.dataset('performance_valorizacao_engajamento')

In [10]:
df['conceito_moral_sexual_rju_lei'].unique()

array(['Apenas conceito de assédio moral no RJU',
       'Sem conceito para assédio no RJU',
       'Apenas conceito de assédio sexual em lei esparsa',
       'Apenas conceito de assédio moral em lei esparsa',
       'Conceito de assédio sexual e moral em lei esparsa'], dtype=object)

In [11]:
df = df.rename(columns={"Regime Jurídico Único": "RJU_lei", "Lei esparsa encontrada": "lei_esparsa","Alterações":"alteracoes_RJU"})

In [19]:
df.columns

Index(['UF', 'Sigla_uf', 'proibicao_assedio_moral_sexual_rju_lei',
       'conceito_moral_sexual_rju_lei', 'RJU_lei', 'lei_esparsa',
       'alteracoes_RJU'],
      dtype='object')

In [12]:
schema=[bigquery.SchemaField('UF','STRING',description='Unidade da Federação.'),
bigquery.SchemaField('Sigla_uf','STRING',description='Sigla da Unidade da Federação.'),
bigquery.SchemaField('proibicao_assedio_moral_sexual_rju_lei','STRING',description='Informa se existe a proibição de assédio no estatuto do servidor ou em lei esparsa'),
bigquery.SchemaField('conceito_moral_sexual_rju_lei','STRING',description='Informa se existe o conceito de assédio no estatuto do servidor ou em lei esparsa'),
bigquery.SchemaField('RJU_lei','STRING',description='Número da Lei do RJU'),
bigquery.SchemaField('lei_esparsa','STRING',description='Número da Lei da lei esparsa'),
bigquery.SchemaField('alteracoes_RJU','STRING',description='Link com alterações no RJU')
]

In [13]:
table_ref = dataset_ref.table('Republica_assedio_estatuto') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=b54da51c-dd7e-4733-9dfa-986ae94d32f9>